Principal Files
This data challenge contains one dataset of 2000 training sequences. The main files available are the following ones

* Xtr.csv - the training sequences.
* 
* Xte.csv - the test sequences.
* 
* Ytr.csv - the sequence labels of the training sequences indicating bound (1) or not (0).
* 
Each row of Xtr.csv represents a sequence. Xte.csv contains 1000 test sequences, for which you need to predict. Ytr.csv contains the labels corresponding to the training data, in the same format as a submission file.

In [ ]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score

import os


In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xtr.csv", sep=' ',index_col=0).values

labels=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Ytr.csv", sep=',',index_col=0).values


test_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xte.csv",sep=' ', index_col=0).values


#optional data 

train_op_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xtr_mat100.csv", sep=' ',header=None).values
test_op_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xte_mat100.csv", sep=' ',header=None).values





In [ ]:
labels= np.where(labels==0,-1,1)



In [ ]:
labels

# Data exploring

In [ ]:
print(train_data.shape,test_data.shape)

# optional data view

In [ ]:
print(train_op_data.shape)

In [ ]:
labels_one=[train_op_data[i], labels[i]==1, i in range(len(labels)) ]
labels_zeros=[train_op_data[i], labels[i]==0, i in range(len(labels)) ]

In [ ]:
type(labels_one)

In [ ]:
# def plot_margin(X1_train, X2_train, clf):
        
#         def f(x, w, b, c=0):
#             # given x, return y such that [x,y] in on the line
#             # w.x + b = c
#             return (-w[0] * x - b + c) / w[1]
        
#         pl.plot(X1_train[:,0], X1_train[:,1], "ro")
#         pl.plot(X2_train[:,0], X2_train[:,1], "bo")
#         pl.scatter(clf.sv[:,0], clf.sv[:,1], s=100, c="g")
        
#         # w.x + b = 0
#         a0 = -4; a1 = f(a0, clf.w, clf.b)
#         b0 = 4; b1 = f(b0, clf.w, clf.b)
#         pl.plot([a0,b0], [a1,b1], "k")
        
#         # w.x + b = 1
#         a0 = -4; a1 = f(a0, clf.w, clf.b, 1)
#         b0 = 4; b1 = f(b0, clf.w, clf.b, 1)
#         pl.plot([a0,b0], [a1,b1], "k--")
        
#         # w.x + b = -1
#         a0 = -4; a1 = f(a0, clf.w, clf.b, -1)
#         b0 = 4; b1 = f(b0, clf.w, clf.b, -1)
#         pl.plot([a0,b0], [a1,b1], "k--")
        
#         pl.axis("tight")
#         pl.show()

In [ ]:
plot_margin()

# Split the training dataset in train and validation set

In [ ]:

X_train_mat100 = scale(train_op_data)
# 
X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, labels, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape)


# Hard margin svm with SGD from scratch

In [ ]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [ ]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [ ]:
def sgd(features, outputs,learning_rate=0.01):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 2  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = (features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights -(learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [ ]:
def insert_intercept(train_op_data):
    N=train_op_data.shape[0]
   
    a = np.ones((N,1))
    
    train_op_data=np.append(train_op_data,a, axis=1)
    return train_op_data

In [ ]:
train_op_data=insert_intercept(train_op_data)

In [ ]:
def train(x_train,y_train, learning_rate=7):
    W = sgd(X_train, y_train)
    print("training finished.")
    return W

w=train(X_train,y_train)
   

In [ ]:
def validation(x_val,y_val,w):
    y_test_predicted = np.array([])
    for i in range(x_val.shape[0]):
        yp = np.sign(np.dot(W, x_val[i])) #model
        y_test_predicted = np.append(y_test_predicted, yp)
    return accuracy_score(y_val, y_test_predicted)

In [ ]:
validation(X_val,y_val,w)

In [ ]:

def cross_validate(x_data,y_data,lr,lamda=0.2,epoch=10,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
                
        
          
        w = train(x_train,y_train)
        
        result = validation(x_test,y_test,w)
        aggrigate_result.append(result)
        
        value = sum(aggrigate_result)/len(aggrigate_result)
    return aggrigate_result ,value

In [ ]:
X_train_mat100.shape,labels.shape

In [ ]:
cross_validate(train_op_data,labels,lr=0.01,lamda=0.2,epoch=100,k=5)

# soft margin SVM form scratch

In [ ]:
from numpy import linalg
import cvxopt
import cvxopt.solvers

# Define a set of kernels

In [ ]:
class kernels:
    def linear_kernel(x1, x2):
        return np.dot(x1, x2)

    def polynomial_kernel(x, y, p=3):
        return (1 + np.dot(x, y)) ** p

    def gaussian_kernel(x, y, sigma=5.0):
        return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))